In [1]:
from pathlib import Path
import datetime as dt
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np


OUT_DIR = Path('../output')

In [4]:
def build_roster(year, month, volunteers_by_weekday, meeting_time=None):
    """
    Given a year (integer) and a month (integer), build the roster for
    that year and month as a DataFrame with the columns
    
    - date: date of a shift
    - weekday: weekday of the shift, e.g. 'Sunday'
    - volunteers: names of volunteers working the shift
    - notes: any notes about the shift
    
    Give a dictionary of volunteers by weekday to seed the roster, 
    e.g. ``{'Thursday': ['Tommy', 'Etienne'], 'Sunday': ['?', '?']}``.
    Optionally specify a time for the monthly meeting, e
    e.g. ``('Sunday', 1, '15:00', '16:00')`` for the first Sunday of the month
    during 15:00--16:00.
    """
    # Create a roster for every day of the month
    d1 = dt.date(year, month, 1)
    d2 = d1 + relativedelta(months=1)
    ix = pd.date_range(d1, d2, closed='left')
    f = pd.DataFrame(index=ix)
    f['date'] = f.index.strftime('%Y-%m-%d')
    f['weekday'] = f.index.day_name()
    
    # Keep only the days specified by ``volunteers_by_weekday``
    cond = f['weekday'].isin(volunteers_by_weekday.keys())
    f = f[cond].copy()

    # Assign volunteers
    f['volunteers'] = f['weekday'].map(lambda x:
      ' & '.join(volunteers_by_weekday[x]))
    
    # Add notes, including staff meeting.
    f['notes'] = ''
    if meeting_time is not None:
        cond = f['weekday'] == meeting_time[0]
        cond &= f.index.day >= 7*(meeting_time[1] - 1)
        cond &= f.index.day <= 7*meeting_time[1]
        f.loc[cond, 'notes'] = 'Staff meeting {!s}--{!s}'.format(*meeting_time[2:])
    
    return f.reset_index(drop=True)

def get_year_month(roster, format="%Y%m"):
    """
    Given a roster, return the year and month it pertains to 
    in the given format.
    """
    return pd.to_datetime(roster['date']).dt.strftime(format).iat[0]

def to_markdown(f, path=None):
    """
    Given DataFrame, return its corresponding Markdown table (string).
    If a path is given (string or Path object), then instead save the
    Markdown table to a Markdown file located at the path.
    """
    if path is not None:
        # Make path parent directory if it doesn't exist
        path = Path(path)
        if not path.parent.exists():
            path.parent.mkdir(parents=True)
    
    cols = f.columns

    # Create a new DataFrame with just the markdown table line
    header = pd.DataFrame([['---',]*len(cols)], columns=cols)

    # Create a new concatenated DataFrame
    g = pd.concat([header, f])

    # Save or return Markdown
    if path is not None:
        g.to_csv(path, sep="|", index=False)
    else:
        return g.to_csv(sep="|", index=False)


In [7]:
# Make roster and print for visual check

volunteers_by_weekday = {
    'Sunday': ['?', '?'],
}

roster = build_roster(2021, 1, volunteers_by_weekday)
print(roster)

         date weekday volunteers notes
0  2021-01-03  Sunday      ? & ?      
1  2021-01-10  Sunday      ? & ?      
2  2021-01-17  Sunday      ? & ?      
3  2021-01-24  Sunday      ? & ?      
4  2021-01-31  Sunday      ? & ?      


In [8]:
# Print roster as Markdown for Loomio

roster_md = to_markdown(roster)
print(roster_md)

# # Write Markdown roster to file
# path = OUT_DIR/'rosterio_{!s}'.format(get_year_month(roster))
# to_markdown(roster, path)

date|weekday|volunteers|notes
---|---|---|---
2021-01-03|Sunday|? & ?|
2021-01-10|Sunday|? & ?|
2021-01-17|Sunday|? & ?|
2021-01-24|Sunday|? & ?|
2021-01-31|Sunday|? & ?|

